In [1]:
import openai
import pandas as pd
from tqdm.auto import tqdm
import tiktoken

tqdm.pandas()

/Users/baker_yang/Desktop/workspace/work/customer_feedback_ai/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
AZURE_OPENAI_KEY = ""
AZURE_OPENAI_ENDPOINT = ""
openai.api_key = AZURE_OPENAI_KEY
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_type = 'azure'
openai.api_version = '2023-05-15'  # this may change in the future
DATAPATH = "Restaurant_Reviews.tsv"

In [3]:
def first_message(*input: str) -> str:

    messages = [
        {"role": "system",
         "content": "Assistant is a helper, help restoran chef to summary the feedback of customers dining experience and/or food. Assistant can say idk if it can not identify the reasons."},

        {"role": "user", "content": "This hole in the wall has great Mexican street tacos, and friendly staff."},
        {"role": "assistant", "content": "Customer feel good with Mexican street tacos and friendly staff"},

        {"role": "user",
         "content": "Took an hour to get our food only 4 tables in restaurant my food was Luke warm, Our sever was running around like he was totally overwhelmed."},
        {"role": "assistant",
         "content": "Customer feel not good with took too long to get the food, tables not enough and bad services of server."},

        {"role": "user",
         "content": "Our server was fantastic and when he found out the wife loves roasted garlic and bone marrow, he added extra to our meal and another marrow to go!"},
        {"role": "assistant",
         "content": "Customer feel good with a fantastic services, roasted garlic, bone marrow and extra meal."},

        {"role": "user",
         "content": "He was extremely rude and really, there are so many other restaurants I would love to dine at during a weekend in Vegas."},
        {"role": "assistant", "content": "Customer feel not good with bad services of server."},

        {"role": "user", "content": "I think not again."},
        {"role": "assistant", "content": "idk"},
    ]

    messages = [
        {"role": "system",
         "content": "Assistant is a helper, help restoran chef to summary the feedback of customers dining experience and/or food. Assistant can say idk if it can not identify the reasons. There are 5 feedbacks from customers, please summarize the feedback and use $ at the end of each summarized feedback."},

        {
            "role": "user",
            "content": \
                "/0 I think not again.$"\
                "/1 This hole in the wall has great Mexican street tacos, and friendly staff.$"\
                "/2 Took an hour to get our food only 4 tables in restaurant my food was Luke warm, Our sever was running around like he was totally overwhelmed.$"\
                "/3 Our server was fantastic and when he found out the wife loves roasted garlic and bone marrow, he added extra to our meal and another marrow to go!$"\
                "/4 He was extremely rude and really, there are so many other restaurants I would love to dine at during a weekend in Vegas.$"
                ,
            "role": "assistant",
            "content": \
                "/0 idk.$"\
                "/1 Customer feel good with Mexican street tacos and friendly staff.$"\
                "/2 Customer feel not good with took too long to get the food, tables not enough and bad services of server.$"\
                "/3 Customer feel good with a fantastic services, roasted garlic, bone marrow and extra meal.$"\
                "/4 Customer feel not good with bad services of server.$"
        }
    ]

    # messages = [
    #     {"role": "system",
    #      "content": "Assistant is a helper, help restoran chef to summary the feedback of customers dining experience and/or food. Assistant can say idk if it can not identify the reasons. There are 5 feedbacks from customers, please summarize the feedback and use $ at the end of each summarized feedback."\
    #      "The input format will like this: "\
    #         "[{'0': 'I think not again.'},"\
    #         "{'1': 'This hole in the wall has great Mexican street tacos, and friendly staff.'},"\
    #         "{'2': 'Took an hour to get our food only 4 tables in restaurant my food was Luke warm, Our sever was running around like he was totally overwhelmed.'},"\
    #         "{'3': 'Our server was fantastic and when he found out the wife loves roasted garlic and bone marrow, he added extra to our meal and another marrow to go!'},"\
    #         "{'4': 'He was extremely rude and really, there are so many other restaurants I would love to dine at during a weekend in Vegas.'}]"\
    #     " And your response will like this: "\
    #         "[{'0': 'idk.'},"\
    #         "{'1': 'Customer feel good with Mexican street tacos and friendly staff.'},"\
    #         "{'2': 'Customer feel not good with took too long to get the food, tables not enough and bad services of server.'},"\
    #         "{'3': 'Customer feel good with a fantastic services, roasted garlic, bone marrow and extra meal.'},"\
    #         "{'4': 'Customer feel not good with bad services of server.'}]"\
    #     }
    # ]

    
    for reviews in input:
        messages.append({"role": "user", "content": reviews})

    try:
        response = openai.ChatCompletion.create(
            engine="gpt-35-turbo",  # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
            messages=messages,
            temperature=0,
            top_p=0
        )
        return response['choices'][0]['message']['content']

    except openai.error.InvalidRequestError as e:
        if e.error.code == "content_filter":
            return "[message be filtered]"
        else:
            return "None"


def gen_embedding(input : str):
    deployment_model_name='text-embedding-ada-002' #This will correspond to the custom name you chose for your deployment when you deployed a model.

    # Send a completion call to generate an answer
    response = openai.Embedding.create(
        input=input,
        engine=deployment_model_name
    )

    # print(response)
    embeddings = response['data'][0]['embedding']

    return embeddings


def num_tokens_from_string(string: str, encoding_name="cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
# A+b -> A+5b
# A -> A/5

In [4]:
# first_message("""This place is not worth your time, let alone Vegas.""")
# gen_embedding("""It's Raw!'""")

In [19]:
df = pd.read_csv(DATAPATH,sep='\t', header=None)
df.columns = ["comment", "label"]

df["n_tokens"] = df.comment.apply(lambda x: num_tokens_from_string(x))
df

,comment,label,n_tokens
0,Wow... Loved this place.,1,6
1,Crust is not good.,0,6
2,Not tasty and the texture was just nasty.,0,9
3,Stopped by during the late May bank holiday of...,1,16
4,The selection on the menu was great and so wer...,1,13
...,...,...,...
1005,"I wouldn't trust you running a bath, let alone...",0,14
1006,"The fish is so raw, it is still finding nemo",0,12
1007,"you burnt the chicken so black, the police arr...",0,11
1008,"the mushroom is so raw, it said the princess i...",0,14


In [24]:
feedback = []
import time

In [33]:
BATCH = 5
n = len(feedback)


for i in range(n, 1010, BATCH):
    print(i, end=" ")
    comment = df["comment"].iloc[i:i+BATCH]
    comments = [f"/{i} {comment.iloc[i]}" for i in range(len(comment))]

    fb = first_message(f"""{comments}""")
    fb = fb.split("$")

    if len(fb) == (BATCH+1):
        fb = fb[:BATCH]
    else:
        if len(fb) != BATCH:
            print(i, fb)
            fb = [""] * BATCH
    
    feedback += fb
    time.sleep(4)

950 955 960 965 970 975 980 985 990 995 1000 1005 

In [34]:
len(feedback)

1010

In [35]:
df["feedback"] = feedback
df["feedback"] = df["feedback"].apply(lambda x: x[3:])
df["feedback"] = df["feedback"].apply(lambda x: x.strip(" "))

In [36]:
df_1 = df.copy()
df_1 = df_1[df_1["feedback"]!='']
df_1 = df_1[df_1["feedback"].str.startswith("Customer")]

In [37]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def gen_embedding(input : str):
    deployment_model_name='text-embedding-ada-002' #This will correspond to the custom name you chose for your deployment when you deployed a model.

    # Send a completion call to generate an answer
    response = openai.Embedding.create(
        input=input,
        engine=deployment_model_name
    )

    # print(response)
    embeddings = response['data'][0]['embedding']

    return embeddings

In [38]:
df_1['ada_embedding'] = df_1["feedback"].apply(lambda x: gen_embedding(x))

In [39]:
df_1.to_csv("process.csv", index=False)